In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/train.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

In [0]:
import pandas as pd

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})

# display 
test_df.head()

In [0]:
!pip install allennlp
!pip install nltk
!pip install tqdm

!pip install -U spacy 
!python -m spacy download en_core_web_sm

In [0]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")

START AGAIN

In [0]:
def extract_modal(obj):
    arr = list() # full set of constituents
    modal= list() # only the modal phrases
    past = list() # only the past phrases
    noun = list()
    verb = list()

    def extract(obj, arr):
        
        if isinstance(obj, dict):
          for k, v in obj.items():
            if isinstance(v, (dict, list)):
                extract(v, arr)
              
            if k=='nodeType' and v=='NP':
                arr.append(v)
            if k=='nodeType' and v=='MD':
                arr.append(v)
            if k=='nodeType' and v=='VP':    
                arr.append(v)
            if k=='nodeType' and v=='VBD': # Verb, past tense
                arr.append(v) 
            if k=='nodeType' and v=='ADVP': # Adverb Phrase
                arr.append(v)           
            if k=='nodeType' and v=='RB':   # Adverb 
                arr.append(v)
            if k=='nodeType' and v=='CC':   # Coordinating conjunction
                arr.append(v)
            if k=='nodeType' and v=='IN':   # Preposition or subordinating conjunction 
                arr.append(v)        
            if k=='nodeType' and v=='SBAR': # Clause starting subordinating conjunction
                arr.append(v)
            if k=='nodeType' and v=='SINV': # Inverted declarative sentence, i.e. one in which the subject follows the tensed verb or modal.
                arr.append(v)

        elif isinstance(obj, list):
          for item in obj:
            if isinstance(item, dict):
              if ('would'==item['word'].lower() or 
                  'could'==item['word'].lower() or 
                  'd'==item['word'].lower()
              ):
                phrase = scrap_from(obj, item)
                modal.append(phrase) # only phrase on the right from item

              if 'MD' in arr: 
                arr.remove('MD')
                for f in obj: # re-visit each fragment 
                  if f['word'].lower().lstrip().endswith(' d'):
                    modal.append(obj)
                  if f['word'].lower().lstrip().startswith('would'):
                    modal.append(obj)
                  if f['word'].lower().lstrip().startswith('could'):
                    modal.append(obj)
                  if f['word'].lower().lstrip().startswith('should'):
                    modal.append(obj)
                  if f['word'].lower().lstrip().startswith('might'):
                    modal.append(obj)  

              if 'NP' in arr: 
                arr.remove('NP')
                noun.append(obj)  

              if 'SINV' in arr: # Inverted declarative sentence (eg Had it been...)
                arr.remove('SINV')
                past.append(obj) 

              if 'VP' in arr:
                arr.remove('VP')
                #verb.append(obj) # can subsume 'MD'
                if 'wish' == obj[0]['word'].lower():
                  past.append(obj)  
                if 'should' in item['word'].lower():
                  phrase = scrap_from(obj, item)
                  past.append(phrase)   

              if 'VBD' in arr: 
                arr.remove('VBD')
                if 'were' in obj[0]['word'].lower():
                  past.append(obj)
                if 'was' in obj[0]['word'].lower():
                  past.append(obj)  
                if 'had' in obj[0]['word'].lower():
                  past.append(obj)
          
              if 'RB' in arr: 
                arr.remove('RB')
                if 'even' == obj[0]['word'].lower():
                  past.append(obj)
              #  if 'instead' in obj[0]['word'].lower():
              #    past.append(obj)
              #  if 'whichever' in obj[0]['word'].lower():
              #    past.append(obj)  
              #  if 'whatever' in obj[0]['word'].lower(): 
              #    past.append(obj)
              #  if 'however' in obj[0]['word'].lower(): 
              #    past.append(obj)
              #  if 'regardless' in obj[0]['word'].lower(): 
              #    past.append(obj)
              #  if 'notwithstanding' in obj[0]['word'].lower(): 
              #    past.append(obj) 

              #if 'SBAR' in arr: # seems redundant but keep in case
              #   arr.remove('SBAR')
              #   for s in obj: # re-visit each sentece in the SBAR
              #     if s['word'].lower().lstrip().startswith('if '):
              #       past.append(s)       
              if 'IN' in arr: 
                  arr.remove('IN') 
                  if 'in' == obj[0]['word'].lower():
                    past.append(obj)  
                  if 'if' == obj[0]['word'].lower(): # space after 'if'
                    past.append(obj)
                  if 'with' == obj[0]['word'].lower():
                    past.append(obj) 
                  #if 'for' == obj[0]['word'].lower(): # "for example"
                  #  past.append(obj) 

              if 'CC' in arr: 
                  arr.remove('CC')   
                  if 'but' == obj[0]['word'].lower():
                    past.append(obj)   
                  if 'because' == obj[0]['word'].lower():
                    past.append(obj)  
                  if 'although' == obj[0]['word'].lower():
                    past.append(obj)
            
              #arr.append(item) # uncomment for the full set
              #print(item)

              extract(item, arr)
  
        return arr

    results = extract(obj, arr)

    return modal, past, noun, verb

In [0]:
def scrap_from(obj, item): 
  word = str() 
  prev = str() 
  for f in obj: 
    if f['word'] == item['word']: 
      word = ' ' 
    if len(word) > 0:  
      if (not (
              word=='s' or word=='d' or 
              word=='\'s' or word=='\'d' or
              word=='re' or word=='ve' or word=='nt' or
              word=='\'re' or word=='\'ve' or word=='n\'t') and  
          not (
              prev in '¥$€£&@({[/---...'  or  # after(eg 'S& P')
              word in '^~=+&@)}]/---....,;:*#% m k x' # before(eg '% 5 M')
          )
      ): word = word + ' '
      word = word + f['word']  
      prev = word   
  return word

In [0]:
def separate_children(obj):
    arr = list()

    def separate(obj, arr):
        
        if isinstance(obj, dict):

            for k, v in obj.items():
              if k != 'children':
                if isinstance(v, (dict, list)):
                    separate(v, arr)
                elif k == 'word':      
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                separate(item, arr)
         
        return arr

    results = separate(obj, arr)
    return results

In [0]:
def extract_constituents():
 for k, v in prediction.items():
  if (k=='hierplane_tree'):
    for ke, val in v.items():
      if (ke=='root'):
        for key, value in val.items():
          if (key == 'children'):
            return extract_modal(value)  

In [0]:
def deduplicate(past):
  for p1 in past:
    for p2 in past:
      if p2 in p1 and len(p1) > len(p2):
        past.remove(p2)          
  return past         

In [0]:
def remove_from(past, modal, last=1):
  pst = past.copy()

  for m in sorted(modal,key=len): # in order of longer consequents
    for p in reversed(sorted(past,key=len)): # shorter antecedents
      if p.rstrip() in m and len(m) > len(p): # p is substring of m
        if p in past and len(past) > last: 
          past.remove(p)
      elif m.rstrip() in p and len(p) > len(m):# m is substring of p
        if m in modal and len(modal) > last: # can't be left empty!
          modal.remove(m) 

  for p in pst: # fresh copy
    for m in modal:
      if m.rstrip() in p: # both p = m or p > m
        if p in past and len(past) > last:
          past.remove(p) # remove p{ m }

  return past, modal         

In [0]:
def separate_from(md,pa):
  if pa in md and len(md) > len(pa):
    if md.index(pa) > 0 :
      md = md[0:md.index(pa)]
      #print('|'+md+'{...}')
    else:
      md = md.replace(pa,'')
      #print('|{...}'+md)  
  return md        

In [0]:
from nltk.tokenize import word_tokenize

def simplify(constit):
  if isinstance(constit, str):
    return constit
  gist = list();
  unique = set() 
  for c in constit:
      p = separate_children(c) # list of c['word']s
      for head in sorted(p,key=len):
        #print(head) # head phrase (few words or one char ',')
        tokenized = word_tokenize(head)
        tokens = set(tokenized) #  without duplicates (eg ',')
        if len(unique.intersection(tokens)) <= len(tokens):
          unique.update(tokens) # without duplicate tokens!!
          gist.extend(tokenized)
          #print(gist)

  result = str()
  prev = str()
  for word in gist:
    #print(prev,'-->',word)
    if (
        not (word=='s' or word=='d' or # no ' ' before! 
             word=='\'s' or word=='\'d' or
             word=='re' or word=='ve' or word=='nt' or
             word=='\'re' or word=='\'ve' or word=='n\'t'
        ) and
        not  
        (
          prev in '¥$€£&@({[/---...' or  # no ' ' after
          word in '^~=+&@)}]/---...,;:*#%'  # or before
        )
    ): result = result + ' '
    prev = word
    result = result + word  
  return result

In [0]:
def defragment(past, text):
  prev = str() # previous fragment
  start = -1
  end = 0
  for p in sorted(past,key=len):
    r = p.lstrip()
    r = r.replace('   ',' ')
    r = r.replace('  ',' ')
    r = r.replace(' ,',',') 
    r = r.replace(' .','.')
    r = r.replace('Mr.','Mr. ')
    r = r.replace('Ms.','Ms. ')
    r = r.replace('Mrs.','Mrs. ')
    r = r.replace('.  ','. ') # Mr.
    r = r.replace(' - ','-')
    #r = r.replace(': ',':') #50/50
    r = r.replace(' !','!')
    r = r.replace(' ?','?')
    r = r.replace('$ ','$')
    r = r.replace(' %','%')
    r = r.replace('( ','(')
    r = r.replace(' )',')')
    r = r.replace(' ;',';')
    r = r.replace(' :',':') 
    r = r.replace(' nt','nt')
    r = r.rstrip()
    try: 
      start = text.index(r)
      #print(prev+' | '+r)
      index = text.find(prev+r)
      #print(start, index, end)
      if index==-1 or index >= end and len(word_tokenize(p)) > 1:
        prev = text[end:start]
        end = start + len(r)
      else: 
        past.remove(p) 
    except ValueError:
      print('{'+r+'} not in:\n '+text)

  return past, start, end       

In [0]:
import nltk
nltk.download('punkt')

original = "Perhaps the Beatles couldn't have gone on as it was, but it could have evolved and changed if they'd all been willing to put the work in."

verbs = list()
modal = list()
nounm = list()
past = list() 
nounp = list()
nouns = list()

#texts = original.rsplit(',') # chunck the sentence
#for text in texts:

text = original #.replace(',', ' ')
text = text.replace('\"','') # remove double quotes
text = text.replace('\'','') # remove quotes
#print(text)

prediction = predictor.predict(sentence=text)
mod, pas, nou, ver = extract_constituents()

for v in ver:
  vr = simplify(v) 
  if vr not in verbs:
    verbs.append(vr)

for u in nou:
  nu = simplify(u) 
  if nu not in nouns:
    nouns.append(nu) 

if len(mod)==0: # if no modal('MD') detected 
  for p in pas:
    pa = simplify(p)
    if pa not in past:
        past.append(pa) 

for m in reversed(sorted(mod,key=len)):
  md = simplify(m) # get one string gist
  if len(md)==0:
    continue
  #print('=======================')  
  #print(md)
  
  nom = str()
  for nm in reversed(sorted(nouns,key=len)):  
    if nm in md:   
      nounm.append(nm)
      nouns.remove(nm)
      nom = nm
  
  for p in reversed(sorted(pas,key=len)):
    pa = simplify(p)
    if len(pa)==0:
      continue
    #print('-----------------------')
    #print(pa)

    new = separate_from(md, pa)
    if new != md and len(new) > 0:
       if md in modal: # remove original
          modal.remove(md) 
       md = new 

    new = separate_from(pa, md)
    if new != pa and len(new) > 0:
       if pa in past: # remove original
          past.remove(pa) 
       pa = new 

    if len(nom) > 0:
      new = separate_from(pa, nom)
      if new != pa and len(new) > 0:
        if pa in past: # remove original
            past.remove(pa) 
        pa = new    

    # separate NPs originating with past tense      
    for np in reversed(sorted(nouns,key=len)):
      if np in pa:
        nounp.append(np)
        nouns.remove(np)
       
        #new = separate_from(pa, np)
        #if new != pa and len(new) > 0:
        #  if pa in past: # remove original
        #      past.remove(pa) 
        #  pa = new 
      
    if pa not in past and len(pa) > 0:
      past.append(pa)         

  if md not in modal and len(md) > 0:        
    modal.append(md)
     
verbs = deduplicate(verbs)
nounm = deduplicate(nounm)
nounp = deduplicate(nounp)
modal = deduplicate(modal)
past = deduplicate(past)

result = remove_from(modal, past)
modal = result[0]
past = result[1]
result = remove_from(past, modal)
past = result[0]
modal = result[1]

nounm = remove_from(nounm, modal,0)[0]
nounp = remove_from(nounp, past, 0)[0]
nounm = remove_from(nounm, verbs,0)[0]
nounp = remove_from(nounp, verbs,0)[0]

# only after nouns are removed above
modal = remove_from(modal, nounm)[0]
past = remove_from(past, nounp)[0]

past = defragment(past, text) [0]
modal = defragment(modal,text)[0] 

print(nouns)
print(verbs)
print(nounm)
print(nounp)
print(past)
print(modal)     

In [0]:
import nltk
nltk.download('punkt')

import pandas as pd

from tqdm import tqdm
from time import sleep

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df = test_df.drop(index=0)
  
test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1], 
    'labels': ''
})

count = test_df['id'].count()
print(count)

i = 0
j = 0
modal_df= pd.DataFrame(columns=['id','labels','text'])
past_df = pd.DataFrame(columns=['id','labels','text'])
simple_df = pd.DataFrame(columns=['id','labels','text'])

for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]

  verbs = list()
  modal = list()
  nounm = list()
  past = list() 
  nounp = list()
  nouns = list()

  text = original #.replace(',', ' ')
  text = text.replace('\"','') # remove double quotes
  text = text.replace('\'','') # remove quotes

  prediction = predictor.predict(sentence=text)
  mod, pas, nou, ver = extract_constituents()

  for v in ver:
    vr = simplify(v) 
    if vr not in verbs:
      verbs.append(vr)

  for u in nou:
    nu = simplify(u) 
    if nu not in nouns:
      nouns.append(nu) 

  if len(mod)==0: # if no modal('MD') detected 
    for p in pas:
      pa = simplify(p)
      if pa not in past:
          past.append(pa) 

  for m in reversed(sorted(mod,key=len)):
    md = simplify(m) # get one string gist
    if len(md)==0:
      continue
    #print('=======================')  
    #print(md)
    
    nom = str()
    for nm in reversed(sorted(nouns,key=len)):  
      if nm in md:   
        nounm.append(nm)
        nouns.remove(nm)
        nom = nm
    
    for p in reversed(sorted(pas,key=len)):
      pa = simplify(p)
      if len(pa)==0:
        continue
      #print('-----------------------')
      #print(pa)

      new = separate_from(md, pa)
      if new != md and len(new) > 0:
        if md in modal: # remove original
            modal.remove(md) 
        md = new 

      new = separate_from(pa, md)
      if new != pa and len(new) > 0:
        if pa in past: # remove original
            past.remove(pa) 
        pa = new 

      if len(nom) > 0:
        new = separate_from(pa, nom)
        if new != pa and len(new) > 0:
          if pa in past: # remove original
              past.remove(pa) 
          pa = new    

      # separate NPs originating with past tense      
      for np in reversed(sorted(nouns,key=len)):
        if np in pa:
          nounp.append(np)
          nouns.remove(np)
        
          #new = separate_from(pa, np)
          #if new != pa and len(new) > 0:
          #  if pa in past: # remove original
          #      past.remove(pa) 
          #  pa = new 
        
      if pa not in past and len(pa) > 0:
        past.append(pa)         

    if md not in modal and len(md) > 0:        
      modal.append(md)
      
  verbs = deduplicate(verbs)
  nounm = deduplicate(nounm)
  nounp = deduplicate(nounp)
  modal = deduplicate(modal)
  past = deduplicate(past)

  result = remove_from(modal, past)
  modal = result[0]
  past = result[1]
  result = remove_from(past, modal)
  past = result[0]
  modal = result[1]

  nounm = remove_from(nounm, modal,0)[0]
  nounp = remove_from(nounp, past, 0)[0]
  nounm = remove_from(nounm, verbs,0)[0]
  nounp = remove_from(nounp, verbs,0)[0]

  # only after nouns are removed above
  modal = remove_from(modal, nounm)[0]
  past = remove_from(past, nounp)[0]

  #assert len(modal) > 0 # can be legit (eg: 203585)
  for m in modal:
    m_row = pd.Series(data={
      'id': test_df['id'][ind]+'C'+ str(i),
      'labels':'0',
      'text': m
    }, name = str(i) )
    modal_df = modal_df.append(m_row)

    who = str()
    for n in nounp:
      who = who  +  n + '/'

    what = str()
    for nn in nounm:
      what = what + '/' + nn

    if len(past) > 0:
      for p in past:
        p_row = pd.Series(data={
          'id': test_df['id'][ind]+'A'+ str(j),
          'labels':'0',
          'text': p
        }, name = str(j) )
        past_df = past_df.append(p_row)   
        j = j+1   

        s_row = pd.Series(data={
          'id': test_df['id'][ind]+'A'+ str(j) + 'C'+ str(i),
          'labels':'0',
          'text': who + p + ' | ' + m + what
        }, name = str(i) )
        simple_df = simple_df.append(s_row) 
    else:
      s_row = pd.Series(data={
        'id': test_df['id'][ind]+'A'+ str(j) + 'C'+ str(i),
        'labels':'0',
        'text': who + ' | ' + m + what
      }, name = str(i) )
      simple_df = simple_df.append(s_row)

    i = i+1

  sleep(1)  

past_df.to_csv(prefix+'antecedents.tsv', sep='\t', index=False, header=False)
modal_df.to_csv(prefix+'consequents.tsv', sep='\t', index=False, header=False)
simple_df.to_csv(prefix+'simplified.tsv', sep='\t', index=False, header=False)

In [0]:
!mv /content/antecedents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/consequents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/simplified.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/